# Install compatible versions of dependent libraries

In [ ]:
# Uninstall existing packages that may cause conflicts
!pip uninstall -y tensorflow tensorflow-metadata

# Install specific compatible versions
!pip install tensorflow==2.11.0
!pip install mediapipe==0.10.15
!pip install protobuf==3.20.3

import tensorflow as tf
import mediapipe as mp

print("TensorFlow version:", tf.__version__)
print("MediaPipe version:", mp.__version__)


# Upload dataset

Upload archive.zip to /content/sample_data/

# Unzip the Dataset

In [ ]:
!unzip /content/sample_data/archive.zip -d /content/asl_alphabet

# Detect hand landmarks in training images using mediapipe

Reading the training images and convert it to RGB (MediaPipe expects RGB input) then MediaPipe process the image to find hand landmarks. Images Landmarks' extracted and saved as a flattened NumPy array. Then the output is saved in a specified directory with a unique filename.


The summary file, output_summary.txt, will contain lines that describe whether hands were detected in each image processed from your dataset. Each line will specify the name of the image file and the corresponding detection result.

Single Display image per Class to satisfy the output size limit

87000 images, detected 67520 , 77.6%

In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np
from google.colab.patches import cv2_imshow

# Initialize MediaPipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.2)

# Define directories
train_dir = '/content/asl_alphabet/asl_alphabet_train/asl_alphabet_train/'
output_dir = '/content/asl_alphabet/hand_landmarks/'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# To track the first image processed for each class
saved_classes = set()
output_summary = []

# Process images in the training directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)

    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)

        # Read the image
        image = cv2.imread(image_path)

        # Check if the image is read correctly
        if image is None:
            print(f"Error reading image: {image_path}")
            continue

        # Convert image to RGB for MediaPipe processing
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Process the image to find hands
        results = hands.process(image_rgb)

        # Check if hands are detected
        if results.multi_hand_landmarks:
            # Store the landmarks
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the image for visualization
                mp.solutions.drawing_utils.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Create a list of landmarks
                landmarks = [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]
                landmarks_array = np.array(landmarks).flatten()  # Flatten the landmarks

                # Save landmarks as a numpy array
                output_file = os.path.join(output_dir, f"{class_name}_{image_name}.npy")
                np.save(output_file, landmarks_array)

            # Save the first image of each class with detected hands
            if class_name not in saved_classes:
                print(class_name)  # Add this line to print the class name
                cv2_imshow(image)  # Display the image
                cv2.waitKey(1)  # Show the image briefly without waiting for key press
                saved_classes.add(class_name)  # Mark this class as processed

            output_summary.append(f"{image_name}: Hands detected")

        else:
            output_summary.append(f"{image_name}: No hands detected")

# Write summary to a file
with open('output_summary.txt', 'w') as f:
    for line in output_summary:
        f.write(line + "\n")

# Release the MediaPipe hands object
hands.close()


**Save The Model**

In [ ]:

# Save the trained model
model_save_path = '/content/asl_hand_landmarks_model.h5'
model.save(model_save_path)

print(f"Model saved to {model_save_path}")